# Insecure Lattice

In [729]:
import numpy as np

# Parameters
n = 100  # Dimension of the lattice
q = 970  # Modulus
sigma = 0.1  # Very small error standard deviation

# Generate a random secret vector s
s = np.random.randint(0, q, size=n)

# Generate a random matrix A with a large hidden structure (backdoor)
def generate_backdoored_A(n, q):
    A = np.random.randint(0, q, size=(n, n))
    # Introduce a large hidden structure: set most of the matrix to be identity matrix
    hidden_index = n - 1  # Make the hidden structure almost the entire matrix
    for i in range(hidden_index):
        A[i, i] = 1
    return A

A = generate_backdoored_A(n, q)

# Generate a small error vector e (minimized)
e = np.zeros(n, dtype=int)

# Compute the public key: b = A * s + e
b = (A @ s + e) % q

# Public key: (A, b)
# Private key: s
public_key = (A, b)
private_key = s

print("Public key (A, b):")
print("A =", A)
print("b =", b)
print("\nPrivate key (s):")
print("s =", s)

def encrypt(message, public_key, q=970):
    A, b = public_key
    n = A.shape[0]

    # Convert the message to a vector
    message_bytes = message.encode('utf-8')
    m = np.array([b for b in message_bytes]) % q

    # Generate a controlled random vector r
    r = np.random.randint(0, 2, size=n)

    # Compute the ciphertext
    c1 = (A.T @ r) % q
    c2 = (np.dot(b, r) + m) % q

    return c1, c2

message = "ABC"
ciphertext = encrypt(message, public_key)
print("\nCiphertext:")
print("c1 =", ciphertext[0])
print("c2 =", ciphertext[1])

def decrypt(ciphertext, private_key, q=970):
    c1, c2 = ciphertext
    s = private_key
    n = len(s)

    # Compute the inner product of s and c1
    sc1 = np.dot(s, c1) % q

    # Decrypt the message
    decrypted_message = (c2 - sc1) % q

    # Convert the decrypted vector to a string
    decrypted_text = ''.join([chr(int(c)) for c in decrypted_message if 0 <= c < 128])
    return decrypted_text

decrypted_message = decrypt(ciphertext, private_key)
print("\nDecrypted message:")
print(decrypted_message)


def backdoor_decrypt(ciphertext, public_key, q=970):
    A, b = public_key
    c1, c2 = ciphertext
    n = A.shape[0]

    # Exploit the large hidden structure in A to recover the secret vector s
    hidden_index = n - 1  # Make the hidden structure cover almost the entire matrix
    s_recovered = np.zeros(n, dtype=int)
    for i in range(hidden_index):
        s_recovered[i] = c1[i]
    
    # Use the recovered part of s to decrypt the message
    sc1 = np.dot(s_recovered, c1) % q

    # Decrypt the message using the partially recovered secret
    decrypted_message = (c2 - sc1) % q

    # Convert the decrypted vector to a string
    decrypted_text = ''.join([chr(int(c)) for c in decrypted_message if 0 <= c < 128])
    return decrypted_text

# Attacker decrypts the ciphertext
attacker_decrypted_message = backdoor_decrypt(ciphertext, public_key)
print("\nAttacker Decrypted message:")
print(attacker_decrypted_message)



Public key (A, b):
A = [[  1  51 627 ... 317 558 269]
 [510   1 851 ... 843  58 441]
 [889 833   1 ... 171 924 211]
 ...
 [594 262 288 ...   1 150 301]
 [335 887 283 ... 628   1 853]
 [163 380 925 ... 853 494 314]]
b = [606   4 435 183 702 904 152 404 704 669  27 310 492 372 741 596 397 127
 277 727 230 399 567 689 265 708 563 243  57 919 621 173 415 782  55 384
 957 947 121 672 965  24 227 375 619 542 307  94 477 665  69 624 923 669
 864 640 656 942 627 168 879 502 869 313 756 963 441  53 365 320 297 271
 110 273 845 134 881 514 525 227 803 832 851 651 957 882 359  18 617 442
 908 450 716 300 134 504 258  23 522 466]

Private key (s):
s = [105 441 565 934 352 823 116 913  17 751 877 602 136 475 172 504   3 557
 815  44 591  97 322 878 613 107 387 184 193 732  86 275 787 149 295 757
 654 479 707 881 460 951 490 228 285  27 671 796 646 550  93 806  97  36
 270  53 174 435 406 645 508 727 633  82  56 395 570  77 314 255  78 252
 805 572 260  88  74 869 370 907 298 620 526 170 884 900 570

# Secure Lattice Implementation

In [140]:
import numpy as np

# Parameters
n = 10  # Dimension of the lattice
q = 97  # Modulus
sigma = 3.0  # Error standard deviation

# Generate a random secret vector s
s = np.random.randint(0, q, size=n)

# Generate a random matrix A
A = np.random.randint(0, q, size=(n, n))

# Generate a small error vector e
e = np.random.normal(0, sigma, size=n).astype(int) % q

# Compute the public key: b = A * s + e
b = (A @ s + e) % q

# Public key: (A, b)
# Private key: s
public_key = (A, b)
private_key = s

print("Public key (A, b):")
print("A =", A)
print("b =", b)
print("\nPrivate key (s):")
print("s =", s)

def encrypt(message, public_key, q=97):
    A, b = public_key
    n = A.shape[0]

    # Convert the message to a vector
    message_bytes = message.encode('utf-8')
    m = np.array([b for b in message_bytes]) % q

    # Generate a random vector r
    r = np.random.randint(0, 2, size=n)

    # Compute the ciphertext
    c1 = (A.T @ r) % q
    c2 = (np.dot(b, r) + m) % q

    return c1, c2

message = "HELLO"
ciphertext = encrypt(message, public_key)
print("\nCiphertext:")
print("c1 =", ciphertext[0])
print("c2 =", ciphertext[1])

def decrypt(ciphertext, private_key, q=97):
    c1, c2 = ciphertext
    s = private_key
    n = len(s)

    # Compute the inner product of s and c1
    sc1 = np.dot(s, c1) % q

    # Decrypt the message
    decrypted_message = (c2 - sc1) % q

    # Convert the decrypted vector to a string
    decrypted_text = ''.join([chr(c) for c in decrypted_message if c != 0])
    return decrypted_text

decrypted_message = decrypt(ciphertext, private_key)
print("\nDecrypted message:")
print(decrypted_message)


Public key (A, b):
A = [[31  6 77 15 23 34 52 46  5 36]
 [15 73  6 76 28 74 93 52 55 33]
 [83 23 34 30 18 31 30 90 54 66]
 [72 14 62  8 30 76 85 55 75 63]
 [50 32 92 61 96 71 63 43 71  5]
 [54 11 72 20 26 42 27 79 15 52]
 [68 35 57  5 46 62 82 43 71 16]
 [ 8 61 11 17  2 78 96 92 44 68]
 [10 51 93 96 42 71 19 69 46 82]
 [19  2 83 76 93  7 17 69 38 89]]
b = [40 17 46 43 71 31  4 16 27 76]

Private key (s):
s = [29 10 95 84 87 58 45 50 91 96]

Ciphertext:
c1 = [39 48 87  8 94 58 71 19  6 95]
c2 = [83 80 87 87 90]

Decrypted message:
HELLO
